# 반드시 처음부터 끝까지 스켈레톤 코드를 살펴보고 구현하기 시작하길 바란다

## 1. 스켈레톤 코드를 [복사 및 편집] 하여 사용한다.
## 2. 아래의 [Empty Module 3개]를 직접 구현한다.


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-tp2-spam/train.csv
/kaggle/input/2021-ml-tp2-spam/test.csv
/kaggle/input/2021-ml-tp2-spam/sample_submit.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

- nltk 라이브러리에서 punkt 데이터를 다운 받음, 이 데이터는 영화 리뷰와 같은 문서 데이터로 문자의 tokeninizer를 위해서 필요하다
- nltk 라이브러리를 이용해서 불용어를 다운 받음

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df_train = pd.read_csv("/kaggle/input/2021-ml-tp2-spam/train.csv",encoding="latin-1")
df_test = pd.read_csv("/kaggle/input/2021-ml-tp2-spam/test.csv",encoding="latin-1")
df_train.Data

0       No I'm in the same boat. Still here at my moms...
1       (Bank of Granite issues Strong-Buy) EXPLOSIVE ...
2          They r giving a second chance to rahul dengra.
3          O i played smash bros  &lt;#&gt;  religiously.
4       PRIVATE! Your 2003 Account Statement for 07973...
                              ...                        
4452    I came hostel. I m going to sleep. Plz call me...
4453                               Sorry, I'll call later
4454        Prabha..i'm soryda..realy..frm heart i'm sory
4455                           Nt joking seriously i told
4456                  In work now. Going have in few min.
Name: Data, Length: 4457, dtype: object

In [5]:
X_train = df_train["Data"]
y_train=df_train["Class"]
X_test=df_test["Data"]
X_train.head()

0    No I'm in the same boat. Still here at my moms...
1    (Bank of Granite issues Strong-Buy) EXPLOSIVE ...
2       They r giving a second chance to rahul dengra.
3       O i played smash bros  &lt;#&gt;  religiously.
4    PRIVATE! Your 2003 Account Statement for 07973...
Name: Data, dtype: object

# [Empty Module #1] 텍스트 데이터 전처리  

목표: 텍스트 데이터를 처리하기 위한 여러 과정을 거쳐, 머신을 위한 데이터를 만든다. 

```
[input]
--------------
- text: 텍스트 문장 데이터 

[output]
--------------
- text: 전처리를 완료한 문장 데이터 
    
```

In [6]:
# ------------------------------------------------
# [Empty Module #1] 텍스트 데이터 전처리
# ------------------------------------------------
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer

def data_processing(text):
    # ------------------------------------------------------------
    # 구현 가이드라인 
    # ------------------------------------------------------------
    # [1] re.sub 사용해 text 속 '[^A-Za-z]' 외의 문자만을 찾아내 제거한후, pre_words 변수에 저장
    #      1) pattern은 '[^A-Za-z]', repl=' ' 로 각각 설정.
    #      2) 이모지나 숫자,점과 같은 문자외의 것들을 제거했다. (이모지는 감정 분석과 관련해서 몇가지 의미를 나타내지만 이 테스크에서는 이러한 의미도 제거함.)
    #
    # [2] pre_words의 lower 내장 함수를 이용해 대문자들은 소문자로 변경
    #      1)  대, 소문자가 구분되어 있으면 "Go"와 "go" 와 같이 동일한 단어를 머신은 다른 단어로 취급한다. 따라서 대문자를 모두 소문자로 변경.
    #
    # [3] word_tokenize 함수를 이용해 pre_word 를 토큰화하여 word를 리스트화한 후 tokenized_words변수에 저장
    #
    # [4] nltk 라이브러리로 다운 받은 stopwords의 "words" 내장 함수를 이용해 english 불용어를 찾아서 stops 변수에 저장  
    #      1) 불용어: 텍스트 분류에서 불용어는 텍스트의 중요도을 결정하는데 영향을 미치지 않는 단어임. 
    #                    (ex: the, we, a , will), 따라서 불필요한 단어가 예측 모델에 악영향을 끼칠 수 있기 때문에 제거.
    #
    # [5] [3] 에서 찾은 문자열 중 단어가 [4] 에서 찾은 불용어 속에 없을 경우, tokenized_words_remove 리스트에 append 
    #
    # [6] PorterStemmer 속 stem 내장 함수를 이용해 동일 의미를 갖는 단어를 동일한 단어로 변경하는 과정을 거친 후 다시 저장.
    #    
    # ------------------------------------------------------------
    ##############

    # [1] A-Z, a-z를 제외한 문자는 공백으로 만들어 pre_words에 저장
    pre_words = re.sub('[^A-Za-z]',' ', text)

    # [2] pre_words의 문자들을 소문자로 변경
    pre_words = pre_words.lower()

    # [3] pre_words의 문장 토큰화해 tokenized_words에 저장
    tokenized_words = word_tokenize(pre_words)

    # [4] stops에 불용어 set 저장
    stops =set(stopwords.words('english')) 
    
    tokenized_words_remove=[]   
    for w in tokenized_words: 
        # [5] tokenized_words의 단어가 불용어가 아니라면
        if w not in stops: 
            # tokenized_words_remove에 해당 단어 추가
            tokenized_words_remove.append(w)


    stemmer = PorterStemmer()    # 어간추출
    for i in range(len(tokenized_words_remove)):
        # [6] stem 내장 함수를 이용해 어간 추출
        tokenized_words_remove[i] = stemmer.stem(tokenized_words_remove[i])


    return ( " ".join( tokenized_words_remove ))

In [7]:
X_train=[data_processing(i) for i in X_train]
X_test=[data_processing(i) for i in X_test]


# [Empty Module #2] Bag of Word 

목표: 문장 데이터를 머신을 학습하기 위한 실수화된 Feature로 변경하기로한다. 

- train 과 test 데이터 전부 type을 ('U')로 변경하여 Countvectorizer를 사용한다. 
- [설명 링크](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html?highlight=countvectorizer#sklearn.feature_extraction.text.CountVectorizer)

In [8]:
# # ------------------------------------------------
# # [Empty Module #2] 텍스트 데이터 Bag of word  feature  화 
# # ------------------------------------------------
# from sklearn.feature_extraction.text import CountVectorizer
# # ------------------------------------------------------------
# # 구현 가이드라인 
# # ------------------------------------------------------------
# # [1]  CountVectorizer를 정의 
# #           1) max_features를 100으로 지정 
# # [2] X_train 과 X_test를 numpy array로 변환 후 데이터 타입을 "U"로 변경해 저장
# #
# # [3] CountVectorizer를 이용해 X_train은 학습 및 변환을 하고, X_test는 변환을 진행. 
# # ------------------------------------------------------------
# ###########
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer 생성 
vectorizer = CountVectorizer(max_features = 200)
             
# numpy array로 변환 후 데이터 타입을 "U"로 변경해 저장
X_train = np.asarray(X_train).astype("U")
X_test = np.asarray(X_test).astype("U")

# X_train은 학습 및 변환을 하고, X_test는 변환
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

In [9]:
# from sklearn.feature_extraction.text import HashingVectorizer
# #  HashingVectorizer 생성 
# vectorizer = HashingVectorizer()
# #  numpy array로 변환 후 데이터 타입을 "U"로 변경해 저장
# X_train = np.asarray(X_train).astype("U")
# X_test = np.asarray(X_test).astype("U")
# # X_train은 학습 및 변환을 하고, X_test는 변환
# X_train_features = vectorizer.fit_transform(X_train)
# X_test_features = vectorizer.transform(X_test)

In [10]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# # TfidfVectorizer 생성 
# vectorizer = TfidfVectorizer(max_features = 200)
# #  numpy array로 변환 후 데이터 타입을 "U"로 변경해 저장
# X_train = np.asarray(X_train).astype("U")
# X_test = np.asarray(X_test).astype("U")
# # X_train은 학습 및 변환을 하고, X_test는 변환
# X_train_features = vectorizer.fit_transform(X_train)
# X_test_features = vectorizer.transform(X_test)

In [11]:
# spam인 데이터는 1, ham인 데이터는 0 대입
y_train[y_train=="ham"]=0
y_train[y_train=="spam"]=1
y_train=y_train.astype("uint8")

In [12]:
X = X_train_features
y = y_train
test = X_test_features

## [Empty Module #3] SVM: classifier
목표: SVC() 를 활용해 classification 을 진행

- fit()으로 train 에 대한 머신러닝 학습
- predict()으로 test 에 대한 정답을 추론 하여 반환

In [13]:
# 성능 확인을 위한 train-validation set 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=0, stratify = y)

In [14]:
# ------------------------------------------------
# [Empty Module #3] 텍스트 데이터 Bag of word  feature  화 
# ------------------------------------------------
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# [1]  SVC 정의 
#           1) gamma="auto" 사용 
# [2] X_train_features과 y_train으로 SVC 학습진행 후, X_test_features로 predict 진행
#
# ------------------------------------------------------------
###########
# 최적의 성능을 찾기 위해 GridSearch 
params = {
    'C' : [3, 4, 5, 6, 7],
    'kernel' : ['rbf'],
    
}
clf = GridSearchCV(SVC(random_state = 0), params, cv=5)
clf.fit(X, y)
print(clf.best_params_)


{'C': 3, 'kernel': 'rbf'}


In [15]:
from sklearn.svm import SVC

clf = SVC(kernel = 'rbf', C = 5, random_state=0)
clf.fit(X_train, ytrain)
print(clf.score(X_test, ytest))

y_p = clf.predict(X_test)
from sklearn import metrics

print(metrics.confusion_matrix(ytest, y_p))

0.9775784753363229
[[1153    6]
 [  24  155]]


# Predict to CSV

In [16]:
# 예측
clf = SVC(kernel = 'rbf', C = 5, random_state=0)
clf.fit(X, y)
y_pred = clf.predict(test)
df_pred={"ID": range(np.array(y_pred).shape[0]),"Class":y_pred}
df_pred=pd.DataFrame(df_pred)
df_pred.to_csv("baseline_TF.csv",index=False)
